In [ ]:
import os
import numpy as np
from deepface import DeepFace
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
# Dataset folder path
dataset_dir = r"path"

In [ ]:

# Embedding and tag lists
X = []
y = []

print("Data embedding is starting...")

for person_name in os.listdir(dataset_dir):
    person_folder = os.path.join(dataset_dir, person_name)
    if not os.path.isdir(person_folder):
        continue
    
    print(f"Data is being processed for {person_name} ")
    
    for img_name in os.listdir(person_folder):
        img_path = os.path.join(person_folder, img_name)
        try:
            # Extract face embedding
            embedding = DeepFace.represent(img_path, model_name='Facenet')[0]["embedding"]
            X.append(embedding)
            y.append(person_name)
        except Exception as e:
            print(f"Error: Face is not detected for {img_path} or there are another problem: {e}")

X = np.array(X)
y = np.array(y)

print(f"Total {len(X)} photo is processed")


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = SVC(kernel='linear', probability=True)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, y_pred):.2f}")

In [ ]:
import ast

# Read dictionary data from TXT file
def load_person_info(filepath):
    with open(filepath, "r", encoding="utf-8") as file:
        data = file.read()
        return ast.literal_eval(data) 

test_img = r"D:\deneme7.jpg"

try:
    person_info = load_person_info("D:\person_info.txt")

    # Get the embedding and make a prediction
    embedding_test = DeepFace.represent(test_img, model_name='Facenet')[0]["embedding"]
    prediction = model.predict([embedding_test])[0]

    print(f"Person Full Name: {prediction}")

    if prediction in person_info:
        info = person_info[prediction]
        print(f"Student Number: {info['Student Number']}")
        print(f"ID Number: {info['ID']}")
        print(f"Department: {info['Department']}")
        print(f"Date of Birth: {info['Date of Birth']}")
    else:
        print("PERSON INFO NOT FOUND!")

except ValueError as e:
    if "Face could not be detected" in str(e):
        print("FACE NOT DETECTED!")
    else:
        raise